# ADV Example

The following example shows a simple workflow for analyzing ADV data using DOLfYN's tools.

Start by importing the necessary DOLfYN tools:

In [1]:
import dolfyn as dlfn
import dolfyn.adv.api as avm

## Read Raw Instrument Data

DOLfYN currently only carries support for the Nortek Vector ADV. The example here loaded is simple 'tabletop' test data.

In [2]:
dat = dlfn.read_example('vector_data01.VEC') # use `dlfn.read([path to file + filename])` for your data
dat

Reading file c:\users\mcve343\dolfyn\dolfyn\example_data\vector_data01.VEC ...
 end of file at 3000000 bytes.


<xarray.Dataset>
Dimensions:              (dir: 3, earth: 3, inst: 3, time: 122912, x: 3, x*: 3)
Coordinates:
  * time                 (time) float64 1.34e+09 1.34e+09 ... 1.34e+09 1.34e+09
  * dir                  (dir) <U1 'X' 'Y' 'Z'
  * x                    (x) int32 1 2 3
  * x*                   (x*) int32 1 2 3
  * inst                 (inst) <U1 'X' 'Y' 'Z'
  * earth                (earth) <U1 'E' 'N' 'U'
Data variables:
    beam2inst_orientmat  (x, x*) float64 2.709 -1.34 -1.364 ... -0.3438 -0.3499
    c_sound              (time) float32 1.493e+03 1.493e+03 ... nan nan
    heading              (time) float32 5.6 10.5 10.51 10.52 ... nan nan nan nan
    pitch                (time) float32 -31.5 -31.7 -31.69 ... nan nan nan
    roll                 (time) float32 0.4 4.2 4.253 4.306 ... nan nan nan nan
    temp                 (time) float32 12.67 12.68 12.68 12.68 ... nan nan nan
    vel                  (dir, time) float32 -1.002 -1.008 -0.944 ... nan nan
    amp                  (dir, time) uint8 104 110 111 113 108 112 ... 0 0 0 0 0
    corr                 (dir, time) uint8 97 91 97 98 90 95 95 ... 0 0 0 0 0 0
    orientation_down     (time) bool True True True True ... True True True True
    pressure             (time) float64 5.448 5.436 5.484 5.448 ... 0.0 0.0 0.0
    orientmat            (inst, earth, time) float32 0.0832 0.155 ... -0.7065
Attributes:
    config:       {'ProLogID': 149, 'ProLogFWver': '4.08', 'config': 15412, '...
    inst_make:    Nortek
    inst_model:   Vector
    inst_type:    ADV
    rotate_vars:  ['vel']
    freq:         6000
    SerialNum:    VEC 9062
    Comments:     APL-UW vector on Tidal Turbulence Mooring in Admiralty, tim...
    fs:           32.0
    coord_sys:    inst
    has_imu:      0

The loaded dataset contains three primary groups of data. One: the velocity, amplitude, and correlation of the Doppler velocimetry; Two: measurements of the instrument's bearing and environment; and Three: the orientation matrices DOLfYN uses for rotating through coordinate frames.

## QC'ing Data

ADV velocity data tends to have spikes due to Doppler noise, and the common way to "despike" the data is by using the phase-space algorithm by Goring and Nikora (2002). DOLfYN integrates this function using a 2-step approach: create a logical mask where True corresponds to a spike detection, and then utilize an interpolation function to replace the spikes.

In [3]:
# Clean the file using the Goring+Nikora method:
mask = avm.clean.GN2002(dat.vel)
# Replace datapoints via cubic spline interpolation
dat_cln = avm.clean.cleanFill(dat, mask, method='pchip')

print('Percent of data containing spikes: {0:.2f}%'.format(100*mask.mean()))

# # If interpolation isn't desired:
# data.coords['mask'] = (('dir','time'), ~mask)
# data.vel.values = data.vel.where(data.mask)

Percent of data containing spikes: 0.73%


## Coordinate Rotations

Now that the data has been cleaned, the next step is to rotate the velocity data into true East, North, Up coordinates

In [4]:
# First set the magnetic declination
dat_cln = dlfn.set_declination(dat_cln, 10) # 10 degrees East

# Rotate that data from the instrument to earth frame:
dat_earth = dlfn.rotate2(dat_cln, 'earth')

Once in the true ENU frame of reference, we can calculate the principal flow direction for the velocity data and rotate it into the principal coordinates. Note that for long deployments in tidal channels, the principal flow direction can change based on the tide.

In [5]:
dat_earth.attrs['principal_heading'] = dlfn.calc_principal_heading(dat_earth.vel)
dat_princ = dlfn.rotate2(dat_earth, 'principal')

## Averaging Data

The next step in ADV analysis is to average the velocity data into time bins (ensembles). There are two ways to do this.

In [6]:
# Define an averaging object, and create an 'ensembled' data set:
binner = avm.TurbBinner(n_bin=19200, fs=dat_princ.fs, n_fft=4096)
dat_binned = binner(dat_princ)

# Alternatively, the functional version of TrubBinner:
#dat_binned = avm.calc_turbulence(dat_princ, n_bin=19200, fs=dat_princ.fs, n_fft=4096)
dat_binned

<xarray.Dataset>
Dimensions:           (dir: 3, earth: 3, inst: 3, omega: 2048, spec: 3, stress: 3, time: 6, tke: 3)
Coordinates:
  * time              (time) float64 1.34e+09 1.34e+09 ... 1.34e+09 1.34e+09
  * dir               (dir) <U10 'streamwise' 'x-stream' 'vert'
  * inst              (inst) <U1 'X' 'Y' 'Z'
  * earth             (earth) <U1 'E' 'N' 'U'
  * tke               (tke) <U5 "u'u'_" "v'v'_" "w'w'_"
  * stress            (stress) <U5 "u'v'_" "u'w'_" "v'w'_"
  * spec              (spec) <U3 'Suu' 'Svv' 'Sww'
  * omega             (omega) float64 0.04909 0.09817 0.1473 ... 100.5 100.5
Data variables:
    U_std             (time) float64 0.2759 0.06907 0.06485 0.1529 1.206 1.508
    c_sound           (time) float32 1.491e+03 1.486e+03 ... 1.487e+03 1.49e+03
    heading           (time) float32 80.02 37.56 38.04 154.8 246.7 257.8
    pitch             (time) float32 -33.37 -39.23 -39.55 16.04 -30.62 -30.28
    roll              (time) float32 6.154 9.863 7.789 9.134 9.101 35.66
    temp              (time) float32 12.09 10.68 10.05 10.35 10.89 11.89
    vel               (dir, time) float64 -0.0175 -0.1375 ... 0.3931 0.05314
    amp               (dir, time) float64 111.7 116.1 116.6 ... 92.11 63.64
    corr              (dir, time) float64 94.61 91.79 94.41 ... 94.91 69.7 36.54
    orientation_down  (time) float64 1.0 1.0 1.0 1.0 1.0 1.0
    pressure          (time) float64 23.14 46.99 46.99 8.291 2.95 0.1562
    orientmat         (inst, earth, time) float64 0.3272 0.472 ... -0.7017
    tke_vec           (tke, time) float32 0.04275 0.01531 ... 0.7226 1.668
    stress_vec        (stress, time) float32 0.001576 -0.003259 ... 0.07752
    S                 (spec, time, omega) float32 0.02985 0.02211 ... 0.01022
Attributes:
    n_bin:      19200
    n_fft:      4096
    n_fft_coh:  3200

The benefit to using `TurbBinner` is that one has access to all of the velocity and turbulence analysis functions that DOLfYN contains (See the DOLfYN API in the documentation). If basic analysis will suffice, the `calc_turbulence` function is the most convienent. Either option can still utilize DOLfYN's shortcuts:

In [7]:
dat_binned.Veldata.U_mag

<xarray.DataArray 'vel' (time: 6)>
array([0.57594036, 0.68261418, 0.67809315, 0.14413039, 0.10283421,
       0.54733315])
Coordinates:
  * time     (time) float64 1.34e+09 1.34e+09 1.34e+09 ... 1.34e+09 1.34e+09
Attributes:
    units:        m/s
    description:  horizontal velocity magnitude